In [5]:
from beginning_stuff import HF_CACHE

HF_CACHE

'/home/coder/Python_project/MergingUriel/./big_assets/.cache/huggingface'

In [2]:
from merginguriel.utils import get_iso639_3_code_from_name

get_iso639_3_code_from_name(["English", "Spanish", "French", "German", "Chinese"])

['eng', 'spa', 'fra', 'deu', 'zho']

In [5]:
import pycountry

pycountry.languages.lookup("English")

Language(alpha_2='en', alpha_3='eng', name='English', scope='I', type='L')

In [ ]:
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", cache_dir=HF_CACHE, num_labels=2)
model

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

2025-08-25 05:53:31.595796: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# This script calculates a similarity matrix for all languages available in the lang2vec library
# based on a specified linguistic feature set. The resulting matrix is saved to a CSV file.

import lang2vec.lang2vec as l2v
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# --- User Settings ---
# You can change the feature_type to explore different kinds of linguistic similarity.
# Options include: "syntax_wals", "genetic_wals", "phonology_wals", "inventory_wals", "geo"
feature_type = "syntax_wals" 
output_filename = "language_similarity_matrix.csv"
# --------------------

def generate_similarity_matrix():
    """
    Fetches language feature vectors, computes the all-to-all cosine similarity,
    and saves the resulting matrix to a CSV file.
    """
    print("--- Starting Language Similarity Calculation ---")

    # Step 1: Fetch all available language codes from the library.
    print("Step 1: Fetching all available languages...")
    try:
        all_langs = l2v.available_languages()
        print(f"Found {len(all_langs)} languages.")
    except Exception as e:
        print(f"Error fetching language list: {e}")
        return

    # Step 2: Fetch feature vectors for each language individually inside a loop.
    # This is more robust and avoids a bug in the library when fetching all languages at once.
    print(f"Step 2: Fetching '{feature_type}' feature vectors (this will take several minutes)...")
    all_features = {}
    total_langs = len(all_langs)
    for i, lang in enumerate(all_langs):
        try:
            # Fetch features for one language at a time
            features = l2v.get_features([lang], feature_type)
            if lang in features:
                all_features[lang] = features[lang]
            
            # Print a progress update every 500 languages
            if (i + 1) % 500 == 0 or (i + 1) == total_langs:
                print(f"  ... processed {i + 1} / {total_langs} languages.")

        except Exception as e:
            # If a language fails, print a warning and skip it.
            print(f"Warning: Could not fetch features for language '{lang}'. Skipping. Error: {e}")
            continue

    # Step 3: Prepare the data for calculation.
    valid_langs = list(all_features.keys())
    feature_vectors = [all_features[lang] for lang in valid_langs]

    if not feature_vectors:
        print("No valid feature vectors were found. Cannot proceed.")
        return

    print("Step 3: Creating the feature matrix for calculation...")
    feature_matrix = np.array(feature_vectors)

    np.save("feature_matrix.npy", feature_matrix)

    # # Step 4: Calculate the all-to-all similarity matrix.
    # print("Step 4: Calculating the all-to-all similarity matrix...")
    # try:
    #     similarity_matrix = cosine_similarity(feature_matrix)
    #     print("Calculation complete.")
    # except Exception as e:
    #     print(f"Error during similarity calculation: {e}")
    #     return

    # # Step 5: Convert the result into a user-friendly DataFrame.
    # print("Step 5: Creating a DataFrame for easy viewing and saving...")
    # similarity_df = pd.DataFrame(similarity_matrix, index=valid_langs, columns=valid_langs)

    # # Step 6: Save the final matrix to a CSV file.
    # print(f"Step 6: Saving the full matrix to '{output_filename}'...")
    # try:
    #     similarity_df.to_csv(output_filename)
    #     print(f"Successfully saved the similarity matrix to '{output_filename}'")
    # except Exception as e:
    #     print(f"An error occurred while saving the file: {e}")

    # # --- Final Output ---
    # print("\n--- Process Finished ---")
    # print(f"Shape of the final similarity matrix: {similarity_df.shape}")
    # print("\nHere's a small preview of the matrix (top 5x5):")
    # print(similarity_df.iloc[:5, :5])


if __name__ == '__main__':
    generate_similarity_matrix()

# Restart and Start From Scratch

In [ ]:
import lang2vec.lang2vec as l2v
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

output_filename = "language_similarity_matrix.csv"

all_langs = l2v.available_languages()
all_langs = {str(lang) for lang in all_langs if lang is not None}
print(f"{all_langs = }")

In [ ]:
feature_type = "syntax_knn"
all_features = {}
for i, lang in enumerate(all_langs):
    features = l2v.get_features(["mif", "lex"], feature_type)
    print(f"{features = }")
    break

In [ ]:
features = l2v.get_features(list(all_langs), feature_type)
print(features)

In [ ]:
similarity_matrix = cosine_similarity(list(features.values()))
print(similarity_matrix)

In [ ]:
# Remove negative values
similarity_matrix = (similarity_matrix + 1) / 2
print(similarity_matrix)

In [ ]:
similarity_matrix.diagonal().sum() == similarity_matrix.shape[0]

In [ ]:
def sinkhorn_normalize(matrix, iterations=20):
    """
    Iteratively normalizes a matrix to make it doubly stochastic.

    Args:
        matrix (np.ndarray): The input matrix. Must be non-negative.
        iterations (int): The number of iterations to perform.

    Returns:
        np.ndarray: The normalized matrix where rows and columns sum to 1.
    """
    # Make a copy to avoid modifying the original matrix
    result = matrix.copy()
    
    # Add a small epsilon to avoid division by zero if a row/column is all zeros
    epsilon = 1e-9

    for _ in range(iterations):
        # Normalize across rows
        row_sums = result.sum(axis=1, keepdims=True)
        result = result / (row_sums + epsilon)
        
        # Normalize across columns
        col_sums = result.sum(axis=0, keepdims=True)
        result = result / (col_sums + epsilon)
        
    return result

def filter_top_k(matrix, k):
    """
    Filters the matrix to keep only the top-k values in each row.

    Args:
        matrix (np.ndarray): The input similarity matrix.
        k (int): The number of top values to keep for each row.

    Returns:
        np.ndarray: A sparse matrix with only the top-k values per row.
    """
    # Make a copy to avoid modifying the original
    sparse_matrix = matrix.copy()

    if sparse_matrix.diagonal().sum() == sparse_matrix.shape[0]:
        print("Warning: The diagonal of the matrix does consist entirely of ones. We will make it to 0")
        np.fill_diagonal(sparse_matrix, 0)
    
    # Get the number of rows
    num_rows = sparse_matrix.shape[0]
    print(num_rows)

    for i in range(num_rows):
        row = sparse_matrix[i, :]
        
        # Use np.partition to find the k-th largest value without a full sort
        # It's more efficient than np.sort
        # The [::-1] part is to sort in descending order
        # We find the k-th largest value. Note: k-1 because of 0-based indexing.
        if len(row) > k:
            kth_largest_value = np.partition(row, -k)[-k]
            # Set all values smaller than the k-th largest to zero
            row[row < kth_largest_value] = 0
            
    return sparse_matrix

In [ ]:
k = 5
sparse_k_matrix = filter_top_k(similarity_matrix, k)
normalized_matrix = sinkhorn_normalize(sparse_k_matrix, iterations=20)
print(normalized_matrix)

In [ ]:
normalized_df = pd.DataFrame(normalized_matrix, index=list(features.keys()), columns=list(features.keys()))
normalized_df

In [ ]:
# See only non zero entries
normalized_df["mif"][normalized_df["mif"] > 0]

In [ ]:
normalized_df.to_csv(f"sparsed_{output_filename}")

In [ ]:
similarity_df = pd.DataFrame(similarity_matrix, index=list(features.keys()), columns=list(features.keys()))
similarity_df

In [ ]:
similarity_df.to_csv(output_filename)

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np

X = np.array([[0, 0], [1, 1], [2, 2]])
distances = pairwise_distances(X, metric='euclidean')
print(distances)

In [ ]:
cosine_similarity()

In [ ]:
for key, value in features.items():

# List All Models

In [10]:
from huggingface_hub import list_models

bert_models = list_models(filter="xlm", full=True)
bert_models = list(bert_models)
print(f"{len(bert_models) = }")

len(bert_models) = 50


In [11]:
bert_models

[ModelInfo(id='FacebookAI/xlm-clm-ende-1024', author='FacebookAI', sha='a749c3083a9ccd2b4e0daa19f34758b2f6b31d52', created_at=datetime.datetime(2022, 3, 2, 23, 29, 4, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2023, 4, 6, 13, 40, 40, tzinfo=datetime.timezone.utc), private=False, disabled=None, downloads=31, downloads_all_time=None, gated=False, gguf=None, inference=None, inference_provider_mapping=None, likes=0, library_name='transformers', tags=['transformers', 'pytorch', 'tf', 'safetensors', 'xlm', 'fill-mask', 'multilingual', 'en', 'de', 'arxiv:1901.07291', 'arxiv:1910.09700', 'autotrain_compatible', 'endpoints_compatible', 'region:us'], pipeline_tag='fill-mask', mask_token=None, card_data=None, widget_data=None, model_index=None, config=None, transformers_info=None, trending_score=0, siblings=[RepoSibling(rfilename='.gitattributes', size=None, blob_id=None, lfs=None), RepoSibling(rfilename='README.md', size=None, blob_id=None, lfs=None), RepoSibling(rfilename='c

In [ ]:
import pickle

serializable_models = [vars(model) for model in bert_models]


with open("bert_models.pickle", "wb") as f:
    pickle.dump(serializable_models, f)

In [ ]:
import pickle

with open("bert_models.pickle", "rb") as f:
    loaded_models = pickle.load(f)

print(f"Successfully loaded {len(loaded_models)} models.")


In [ ]:
loaded_models

In [ ]:
import sys

sys.path.append("..")
import pickle

with open("../big_assets/bert_models.pickle", "rb") as f:
    bert_models = pickle.load(f)

bert_models


[{'id': 'sentence-transformers/all-MiniLM-L6-v2',
  'author': 'sentence-transformers',
  'sha': 'c9745ed1d9f207416be6d2e6f8de32d1f16199bf',
  'last_modified': datetime.datetime(2025, 3, 6, 13, 37, 44, tzinfo=datetime.timezone.utc),
  'created_at': datetime.datetime(2022, 3, 2, 23, 29, 5, tzinfo=datetime.timezone.utc),
  'private': False,
  'gated': False,
  'disabled': None,
  'downloads': 91616706,
  'downloads_all_time': None,
  'likes': 3790,
  'library_name': 'sentence-transformers',
  'gguf': None,
  'inference': None,
  'inference_provider_mapping': None,
  'tags': ['sentence-transformers',
   'pytorch',
   'tf',
   'rust',
   'onnx',
   'safetensors',
   'openvino',
   'bert',
   'feature-extraction',
   'sentence-similarity',
   'transformers',
   'en',
   'dataset:s2orc',
   'dataset:flax-sentence-embeddings/stackexchange_xml',
   'dataset:ms_marco',
   'dataset:gooaq',
   'dataset:yahoo_answers_topics',
   'dataset:code_search_net',
   'dataset:search_qa',
   'dataset:eli5',


In [ ]:
import sys

sys.path.append("..")
import pickle

with open("../big_assets/bert_models_strict.pickle", "rb") as f:
    bert_models_strict = pickle.load(f)


In [ ]:
bert_models_strict[2]

{'id': 'google-bert/bert-base-uncased',
 'author': 'google-bert',
 'sha': '86b5e0934494bd15c9632b12f734a8a67f723594',
 'last_modified': datetime.datetime(2024, 2, 19, 11, 6, 12, tzinfo=datetime.timezone.utc),
 'created_at': datetime.datetime(2022, 3, 2, 23, 29, 4, tzinfo=datetime.timezone.utc),
 'private': False,
 'gated': False,
 'disabled': None,
 'downloads': 52870511,
 'downloads_all_time': None,
 'likes': 2386,
 'library_name': 'transformers',
 'gguf': None,
 'inference': None,
 'inference_provider_mapping': None,
 'tags': ['transformers',
  'pytorch',
  'tf',
  'jax',
  'rust',
  'coreml',
  'onnx',
  'safetensors',
  'bert',
  'fill-mask',
  'exbert',
  'en',
  'dataset:bookcorpus',
  'dataset:wikipedia',
  'arxiv:1810.04805',
  'license:apache-2.0',
  'autotrain_compatible',
  'endpoints_compatible',
  'region:us'],
 'pipeline_tag': 'fill-mask',
 'mask_token': None,
 'trending_score': 8,
 'card_data': None,
 'widget_data': None,
 'model_index': None,
 'config': None,
 'transfor

# Preprocess the Sparsed Language Similarity Matrix

In [2]:
import pandas as pd

df = pd.read_csv("../big_assets/sparsed_language_similarity_matrix.csv", index_col=0)
print(df.head())

     mif  lex  sbc  tzb  bli  ayc  cpg  tpt  mam  dbb  ...  bdj  kek  kor  \
mif  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
lex  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
sbc  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
tzb  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
bli  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

     bkd  cwa  adr  gbp  ewo  jac  ttr  
mif  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
lex  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
sbc  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
tzb  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
bli  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 4005 columns]


In [1]:
from merginguriel.preprocessing import find_rows_with_languages_in_set

In [ ]:
find_rows_with_languages_in_set(df, {"", "spa", "fra", "deu", "zho"})

,mif,lex,sbc,tzb,bli,ayc,cpg,tpt,mam,dbb,...,bdj,kek,kor,bkd,cwa,adr,gbp,ewo,jac,ttr


In [14]:
from merginguriel.utils import get_iso639_3_code_from_name

get_iso639_3_code_from_name(["German"])

['deu']

In [4]:
from merginguriel.utils import get_language_name_from_iso639_3

get_language_name_from_iso639_3(['kri', 'pcm', 'enm', 'ang', 'ydd'])

['Krio',
 'Nigerian Pidgin',
 'Middle English (1100-1500)',
 'Old English (ca. 450-1100)',
 'Eastern Yiddish']

In [3]:
from merginguriel.utils import get_all_positive_columns

get_all_positive_columns("eng", df)

['kri', 'pcm', 'enm', 'ang', 'ydd']

In [1]:
import merginguriel

In [5]:
import os

os.environ["DATA_SEED"]

'3147'

In [15]:
# Isolate the row you're interested in
mif_row = df.loc['deu']

# Find ALL columns in that row that are > 0
all_positive_columns = mif_row[mif_row > 0].index.tolist()

# Print them out
print(all_positive_columns)

['ksh', 'yih', 'wep', 'cim', 'nld']


In [ ]:
import pandas as pd
import numpy as np

def find_rows_with_features_in_set(df: pd.DataFrame, feature_list: list) -> pd.DataFrame:
    allowed_features_set = set(feature_list)

    mask = df.apply(lambda row: set(row[row > 0].index).issubset(allowed_features_set), axis=1)

    return df[mask]

In [2]:
!uv pip list | grep auto

Using Python 3.10.18 environment at: /home/coder/Python_project/MergingUriel/.venv
auto-merge-llm           0.1.0       /home/coder/Python_project/MergingUriel/submodules/auto-merge-llm


In [1]:
import auto_merge_llm

# DF

In [4]:
import pandas as pd

df = pd.read_csv("../big_assets/language_similarity_matrix.csv", index_col=0)

In [7]:
df["eng"]["ind"]

np.float64(0.7265477421488705)

In [12]:
from huggingface_hub import list_models

# Define the base model you want to search for
base_model_id = "FacebookAI/xlm-roberta-base"

print(f"Searching for models fine-tuned on: {base_model_id}\n")

# list_models returns a generator, which is memory-efficient
finetuned_models_generator = list_models(
    filter=f"base_model:{base_model_id}"
)

# Iterate through the generator and print the model IDs
count = 0
for model in finetuned_models_generator:
    print(model.modelId)
    count += 1

print(f"\nFound {count} models.")

Searching for models fine-tuned on: FacebookAI/xlm-roberta-base

TigerTrading/TradingBot
KoichiYasuoka/xlm-roberta-base-english-upos
s-nlp/xlmr_formality_classifier
coldfir3/xlm-roberta-base-finetuned-panx-all
coldfir3/xlm-roberta-base-finetuned-panx-de-fr
coldfir3/xlm-roberta-base-finetuned-panx-fr
coldfir3/xlm-roberta-base-finetuned-panx-it
d4niel92/xlm-roberta-base-finetuned-marc-en
papluca/xlm-roberta-base-language-detection
rzsgrt/xlm-roberta-base-finetuned-marc-en
saattrupdan/employment-contract-ner-da
saattrupdan/xlmr-base-texas-squad-da
saattrupdan/xlmr-base-texas-squad-de
saattrupdan/xlmr-base-texas-squad-es
prakod/en-hi-pos-tagger-symcom
saattrupdan/job-listing-relevance-model
saattrupdan/job-listing-filtering-model
triet1102/xlm-roberta-base-finetuned-panx-de
arrandi/xlm-roberta-base-finetuned-panx-de
bousejin/xlm-roberta-base-finetuned-panx-it
haesun/xlm-roberta-base-finetuned-panx-de
haesun/xlm-roberta-base-finetuned-panx-de-fr
haesun/xlm-roberta-base-finetuned-panx-fr
hae

In [4]:
from transformers import AutoModel

model_name_or_path = "gpt2"
model = AutoModel.from_pretrained(model_name_or_path)
model


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [ ]:
# Inference
